## Generate images for evaluation

In [5]:
import os,json
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

import sys
utils_path = os.path.abspath(os.path.join('../..'))
sys.path.append(utils_path)

import torch
from diffusers import DiffusionPipeline, UNet2DConditionModel
from transformers import CLIPTextModel
from utils.dataset_info import get_subjects_prompts_info

output_path = "../../outputs/subjects/dreambooth"

embedding_folds = os.listdir("../../logs/subjects/dreambooth/")

dataset_info_path = "../../pcs_dataset/info.json"

prompts_info, prompts = get_subjects_prompts_info(dataset_info_path)


for subject in embedding_folds:

    # Load the pipeline with the same arguments (model, revision) that were used for training
    model_id = "runwayml/stable-diffusion-v1-5"
    unet = UNet2DConditionModel.from_pretrained(f"../../logs/subjects/dreambooth/{subject}/checkpoint-500/unet")

    # if you have trained with `--args.train_text_encoder` make sure to also load the text encoder
    # text_encoder = CLIPTextModel.from_pretrained("/sddata/dreambooth/daruma-v2-1/checkpoint-100/text_encoder")
    pipeline = DiffusionPipeline.from_pretrained(model_id, unet=unet, dtype=torch.float16, use_safetensors=True)
    pipeline.to("cuda")

    for idx, prompt in enumerate(prompts[prompts_info[subject]["prompt_type"]]):
        prompt = prompt.replace("{0}", "sks")
        prompt = prompt.replace("{1}", f"{prompts_info[subject]['class']}")
        print("prompt: ", prompt)

        image = pipeline(prompt, num_inference_steps=50, num_images_per_prompt=4).images

        save_path = os.path.join(output_path, subject, f"prompt{idx}")
        os.makedirs(save_path, exist_ok=True)

        for img_idx, img in enumerate(image):
            img.save(os.path.join(save_path, f"{img_idx:04d}.jpg"))
    
    print(f"Finished dreambooth in subject: {subject}!")

Keyword arguments {'dtype': torch.float16} are not expected by StableDiffusionPipeline and will be ignored.
Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00, 14.66it/s]


prompt:  a sks statue in the jungle


100%|██████████| 50/50 [00:49<00:00,  1.02it/s]


prompt:  a sks statue in the snow


100%|██████████| 50/50 [00:48<00:00,  1.02it/s]


...

100%|██████████| 50/50 [00:22<00:00,  2.20it/s]


prompt:  a cube shaped sks statue


100%|██████████| 50/50 [00:22<00:00,  2.20it/s]


Finished dreambooth in subject: elephant!


OSError: We couldn't connect to 'https://huggingface.co' to load this model, couldn't find it in the cached files and it looks like ../../logs/subjects/dreambooth/clock2/checkpoint-500/unet is not the path to a directory containing a config.json file.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/diffusers/installation#offline-mode'.

## Evaluation

Calculate img to img similarity and text to img similarity by CLIP Evaluator

In [14]:
import os, sys, json

os.environ["CUDA_VISIBLE_DEVICES"] = "5"

utils_path = os.path.abspath(os.path.join('../..'))
sys.path.append(utils_path)

import numpy as np
from utils.clip_eval import CLIPEvaluator, evaluate_i2i, evaluate_t2i
import re
from utils.dataset_info import get_subjects_prompts_info

In [15]:
def extract_id(folder_name):
    match = re.search(r'\d+', folder_name)
    if match:
        return int(match.group())
    return None

def convert_to_native(data):
    if isinstance(data, np.ndarray):
        return data.tolist()
    elif isinstance(data, np.generic):
        return data.item()
    elif isinstance(data, dict):
        return {key: convert_to_native(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [convert_to_native(item) for item in data]
    else:
        return data

In [16]:
outputs_path = "../../outputs/subjects/dreambooth"
eval_res_path = "../../eval_results/subjects/dreambooth"
dataset_path = "../../pcs_dataset/subjects"
dataset_info_path = "../../pcs_dataset/info.json"

prompts_info, prompts = get_subjects_prompts_info(dataset_info_path)

subjects_list = os.listdir(outputs_path)

evaluation_res = dict()
for subject in subjects_list:
    print("process subject: ", subject)

    res_for_each_subject =dict()

    for sub_dir in os.listdir(os.path.join(outputs_path, subject)):
        print("prompt: ", sub_dir)

        prompt = prompts[prompts_info[subject]["prompt_type"]][extract_id(sub_dir)]
        prompt = prompt.replace("{0} {1}", prompts_info[subject]["class"])
        res_for_each_prompt =dict()

        for generate_img_name in os.listdir(os.path.join(outputs_path, subject, sub_dir)):
            generate_img_path = os.path.join(outputs_path, subject, sub_dir, generate_img_name)
            res_for_each_prompt[generate_img_name] = [evaluate_i2i(generate_img_path, os.path.join(dataset_path, subject)), evaluate_t2i(generate_img_path, prompt)]
        
        res_for_each_subject[prompt] = res_for_each_prompt

        print(res_for_each_prompt)

    evaluation_res[subject] = res_for_each_subject

os.makedirs(eval_res_path, exist_ok=True)

with open(os.path.join(eval_res_path, "evaluation_results.json"), "w") as json_file:
    json.dump(convert_to_native(evaluation_res), json_file, indent=4)

process subject:  mug_skulls
prompt:  prompt13
{'0003.jpg': [0.617, 0.3213], '0000.jpg': [0.545, 0.3525], '0002.jpg': [0.545, 0.3464], '0001.jpg': [0.588, 0.3115]}
prompt:  prompt8
{'0003.jpg': [0.659, 0.3003], '0000.jpg': [0.746, 0.2094], '0002.jpg': [0.5645, 0.3262], '0001.jpg': [0.479, 0.3145]}
prompt:  prompt17
{'0003.jpg': [0.5034, 0.2778], '0000.jpg': [0.626, 0.269], '0002.jpg': [0.6157, 0.2416], '0001.jpg': [0.5137, 0.2734]}
prompt:  prompt19
{'0003.jpg': [0.835, 0.229], '0000.jpg': [0.7646, 0.2227], '0002.jpg': [0.843, 0.2148], '0001.jpg': [0.8057, 0.2192]}
prompt:  prompt10
{'0003.jpg': [0.515, 0.3506], '0000.jpg': [0.397, 0.2666], '0002.jpg': [0.4565, 0.3555], '0001.jpg': [0.634, 0.2573]}
prompt:  prompt2
{'0003.jpg': [0.538, 0.313], '0000.jpg': [0.59, 0.2776], '0002.jpg': [0.6353, 0.2905], '0001.jpg': [0.498, 0.2969]}
prompt:  prompt15
{'0003.jpg': [0.5586, 0.3286], '0000.jpg': [0.549, 0.3342], '0002.jpg': [0.5645, 0.2896], '0001.jpg': [0.6533, 0.3386]}
prompt:  prompt22
{'0